In [1]:
import pandas as pd
import re
import os

In [2]:
# Define your data directory
data_dir = r'f1_wc'
race_events_dir = r'f1_re'

# Load data
lap_times = pd.read_csv(os.path.join(data_dir, 'lap_times.csv'))
pit_stops = pd.read_csv(os.path.join(data_dir, 'pit_stops.csv'))
results = pd.read_csv(os.path.join(data_dir, 'results.csv'))
circuits = pd.read_csv(os.path.join(data_dir, 'circuits.csv'))
safety_cars = pd.read_csv(os.path.join(race_events_dir, 'safety_cars.csv'))
races = pd.read_csv(os.path.join(data_dir, 'races.csv'))
drivers = pd.read_csv(os.path.join(data_dir, 'drivers.csv'))
wiki_data = pd.read_html("https://en.wikipedia.org/wiki/List_of_Formula_One_circuits")[2]

# Drop unnecessary column in the Wiki dataset
wiki_data.drop('Map', axis=1, inplace=True)

In [3]:
# Sort the lap times and pit stops data
lap_times = lap_times.sort_values(['raceId', 'driverId', 'lap'])
pit_stops = pit_stops.sort_values(['raceId', 'driverId', 'stop'])

# Merge lap_times and pit_stops into merge1
merge1 = pd.merge(lap_times, pit_stops, 
                  left_on=['raceId', 'driverId', 'lap'], 
                  right_on=['raceId', 'driverId', 'lap'], 
                  how='left',
                  suffixes=('_lap', '_pit'))

In [4]:
# Replace missing position values in results and convert position to integer
results['position'].replace({'\\N':-1}, inplace=True)
results['position'].astype(str).astype(int)

0         1
1         2
2         3
3         4
4         5
         ..
25835    16
25836    17
25837    18
25838    19
25839    -1
Name: position, Length: 25840, dtype: int32

In [5]:
# Merge merge1 and results into merge2
merge2 = pd.merge(merge1, results, 
                  left_on=['raceId', 'driverId', 'lap'], 
                  right_on=['raceId', 'driverId', 'laps'], 
                  how='left')

In [6]:
# Split Race into Year and Race in safety_cars dataset
safety_cars['Race'] = safety_cars['Race'].str.strip()
safety_cars[['Year', 'Race']] = safety_cars['Race'].str.split(' ', 1, expand=True)

def split_years(season):
    years = []
    if ',' in season:  # case of comma-separated years
        elements = season.split(', ')
        for el in elements:
            el = re.sub(r'\[.*\]', '', el)  # strip off suffixes
            if '–' in el:  # case of a range within comma-separated years
                start, end = map(int, el.split('–'))
                years.extend(list(range(start, end + 1)))
            else:  # case of a single year within comma-separated years
                years.append(int(el))
    elif '–' in season:  # case of a range
        season = re.sub(r'\[.*\]', '', season)  # strip off suffixes
        start, end = map(int, season.split('–'))
        years = list(range(start, end + 1))
    else:  # case of a single year
        season = re.sub(r'\[.*\]', '', season)  # strip off suffixes
        years = [int(season)]
    
    return years

# Apply function and create new column
wiki_data['Year'] = wiki_data['Season(s)'].apply(split_years)

# Expand the list of years into multiple rows
wiki_data = wiki_data.explode('Year')

# Convert 'Year' back to integer
wiki_data['Year'] = wiki_data['Year'].astype(int)

# Drop 'Season(s)' column
wiki_data.drop(['Season(s)'], axis=1, inplace=True)


In [7]:
# Split the 'Grands Prix' column at commas and create a list
wiki_data['Grands Prix'] = wiki_data['Grands Prix'].str.split(',')
wiki_data = wiki_data.explode('Grands Prix')
wiki_data['Grands Prix'] = wiki_data['Grands Prix'].str.strip()


In [8]:
safety_cars['Year'] = safety_cars['Year'].astype(int)
safety_cars['Race'] = safety_cars['Race'].str.lower()
wiki_data['Grands Prix'] = wiki_data['Grands Prix'].str.lower()

In [9]:
# Merge safety_cars and wiki_data into merge3
merge3 = pd.merge(safety_cars, wiki_data, left_on=['Race', 'Year'], 
                  right_on=['Grands Prix', 'Year'], how='left')

In [10]:
# Drop unnecessary columns from races dataset and make race names lower case
races.drop(columns=['url', 'fp1_date', 'fp1_time', 'fp2_date', 'fp2_time', 'fp3_date', 
                    'fp3_time', 'quali_date', 'quali_time', 'sprint_date', 'sprint_time', 'time'], inplace=True)
races['name'] = races['name'].str.lower()

In [11]:
# Merge merge3 and races into merge4
merge4 = pd.merge(merge3, races, left_on=['Race', 'Year'],
                  right_on=['name', 'year'], how='left')

In [12]:
# Merge merge2 and merge4 into merge5
merge5 = pd.merge(merge2, merge4, left_on=['raceId'],
                 right_on=['raceId',], how='left')

In [13]:
# Drop unnecessary columns from drivers dataset
drivers.drop(columns=['url', 'dob'], axis=1, inplace=True)

In [14]:
# Merge final and drivers
final = pd.merge(merge5, drivers, left_on=['driverId'], right_on=['driverId'], how='left')

In [15]:
# Rename position column
final.rename(columns={'position_x': 'position in lap'}, inplace=True)

# Fill missing values for pit stops related columns
final['stop'] = final['stop'].fillna('no pitstop')
final['time_pit'] = final['time_pit'].fillna('no pitstop')
final['duration'] = final['duration'].fillna('no pitstop')
final['milliseconds_pit'] = final['milliseconds_pit'].fillna('no pitstop')

In [16]:
# export final df as final.csv to allow for the creation of the KG in knowledge_graph2.ipynb
final.to_csv('final.csv')

In [41]:
# def extract_year_data(df, years):
#     dataframes = []  # Initialize list to hold dataframes
#     for year in years:
#         # Filter data for specific year
#         year_data = df[df['year'] == year]
#         dataframes.append(year_data)  # Append the dataframe to the list

#         # Save data to csv
#         filename = f'f1_data_{year}.csv'
#         year_data.to_csv(filename, index=False)
        
#     # Concatenate all the dataframes in the list
#     all_data = pd.concat(dataframes, ignore_index=True)
    
#     return all_data


In [42]:
# data_ = extract_year_data(final, [2018, 2019, 2020, 2021, 2022])